# 02 Surgeon Visual
This notebook demonstrates the 'Smart Carving' capabilities of the BoxCarver (The Surgeon). It shows how the carver respects Periodic Boundary Conditions (PBC) and heals clusters by including bonded neighbors.


In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from ase import Atoms
from ase.visualize.plot import plot_atoms

# Add src to path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

from src.carvers.box_carver import BoxCarver


In [ ]:
# Create a simple system with a bond crossing the periodic boundary
# We place two atoms at opposite ends of the box in X direction.
# Box size = 6.0 A
# Atom 1 at x=0.5
# Atom 2 at x=5.5
# Distance = 1.0 A (across boundary)
atoms = Atoms('Cu2', positions=[[0.5, 3.0, 3.0], [5.5, 3.0, 3.0]], cell=[6.0, 6.0, 6.0], pbc=True)

# Visualize original system
fig, ax = plt.subplots()
plot_atoms(atoms, ax, rotation='0x,0y,0z')
ax.set_title("Original System (PBC)")
plt.show()


In [ ]:
# Initialize BoxCarver centered on the first atom (index 0)
# We define a box of size 3.0 A around it.
# Geometrically, the atom at 5.5 is outside the box [0.5 +/- 1.5] -> [-1.0, 2.0].
# However, due to PBC wrapping, 5.5 wraps to -0.5, which IS inside [-1.0, 2.0].
# Smart Carving should handle this.

carver = BoxCarver(atoms, center_index=0, box_vector=3.0)
print("BoxCarver initialized.")


In [ ]:
# Carve with skin_depth=1 to ensure we include bonded neighbors
# In this case, even the geometric cut might catch it due to wrapping, 
# but skin_depth ensures connectivity is respected.
cluster = carver.carve(skin_depth=1)

print(f"Carved cluster contains {len(cluster)} atoms.")
print(f"Cluster positions:
{cluster.positions}")


In [ ]:
# Visualize the carved cluster
# The cluster is non-periodic (vacuum cluster).
# We should see both atoms close to each other.

fig, ax = plt.subplots()
plot_atoms(cluster, ax, radii=0.5, rotation='0x,90y,0z')
ax.set_title("Carved Cluster (Vacuum)")
plt.savefig("surgeon_visual.png")
plt.show()


In [ ]:
# Verification
expected_atoms = 2
if len(cluster) == expected_atoms:
    print("SUCCESS: The surgeon successfully carved the atom and its neighbor across the PBC.")
else:
    print(f"FAILURE: Expected {expected_atoms} atoms, got {len(cluster)}.")

# Check distance in cluster
dist = np.linalg.norm(cluster.positions[0] - cluster.positions[1])
print(f"Distance between atoms in cluster: {dist:.2f} A")
if np.isclose(dist, 1.0):
     print("SUCCESS: Geometry preserved (distance ~ 1.0 A).")
else:
     print(f"WARNING: Distance {dist} != 1.0 A (might have been shifted/wrapped differently but should be consistent).")
